<a href="https://colab.research.google.com/github/antonstorozhilov/sba_experiment/blob/master/SBA_Lending_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Objective: Develop target list of companies to explore further. 
###### Traingulating:
- Growth
- Age
- Business health

## Configuration

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
data_dir = 'drive/My Drive/sba-experiment/data'

In [0]:
import pandas as pd
import numpy as np

In [11]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', 400)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.options.display.float_format = '{:,}'.format
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.float_format = '{:,.4f}'.format

# Lending data unification and cleanup

### Loading Data

In [0]:
# lending_data_504 = pd.read_csv("data/FOIA - 504 (FY1991-Present).csv", encoding = "ISO-8859-1")

In [0]:
lending_data_7a_91t99 = pd.read_csv(data_dir+"/FOIA - 7(a)(FY1991-FY1999).csv")

In [0]:
lending_data_7a_00t09 = pd.read_csv(data_dir+"/FOIA - 7(a)(FY2000-FY2009).csv", encoding="ISO-8859-1")

In [0]:
lending_data_7a_10tPR = pd.read_csv(data_dir+"/FOIA - 7(a)(FY2010-Present).csv")

In [0]:
dict_7a = pd.read_csv(data_dir+"/7a_FOIA_Data_Dictionary.csv")

In [0]:
naics_codes = pd.read_csv(data_dir+"/naics-codes.csv", header=None)

### Cleanup

In [0]:
lending_data_7a_91t99.drop(['AsOfDate', 'Program'], axis=1, inplace=True)
lending_data_7a_00t09.drop(['AsOfDate', 'Program'], axis=1, inplace=True)
lending_data_7a_10tPR.drop(['AsOfDate', 'Program'], axis=1, inplace=True)

In [0]:
dict_7a.columns = ['Field_Name', "Definition"]

In [0]:
naics_codes.columns = ['code', 'desc']

In [0]:
lending_data_7a_91t99 = pd.merge(lending_data_7a_91t99, naics_codes, left_on="NaicsCode", right_on='code')
lending_data_7a_00t09 = pd.merge(lending_data_7a_00t09, naics_codes, left_on="NaicsCode", right_on='code')
lending_data_7a_10tPR = pd.merge(lending_data_7a_10tPR, naics_codes, left_on="NaicsCode", right_on='code')

In [0]:
lending_data_7a_91t99['group'] = '1991-1999'
lending_data_7a_00t09['group'] = '2000-2009'
lending_data_7a_10tPR['group'] = '2010-2020'

In [0]:
lend_df = pd.concat([lending_data_7a_91t99,lending_data_7a_00t09,lending_data_7a_10tPR])

In [0]:
lend_df = lend_df.reset_index(drop=True)

In [0]:
lend_df.to_csv("data/lending_data_7a_1991-2020.csv")

# Loading Clean Data

In [12]:
lend_df = pd.read_csv(data_dir+"/lending_data_7a_1991-2020.csv").drop("Unnamed: 0", axis=1)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
dict_7a = pd.read_csv(data_dir+"data/7a_FOIA_Data_Dictionary.csv")

FileNotFoundError: ignored

# 0. Overall observations

In [15]:
# Number of unique businesses in list (by name)
len(lend_df.BorrName.unique())

986999

In [16]:
# Number of busiensses that have more than one loan
len(lend_df[lend_df.duplicated('BorrName',keep=False)])

349767

# 1. Filtering companies using location, lending amts, headcount, etc.

# 2. How do interest rates, volume, etc. differ by groupings

In [0]:
import scipy.stats as stats

In [0]:
def percentile(n):
    def percentile_(x):
        return x.quantile(n)
    percentile_.__name__ = 'percentile_{:2.0f}'.format(n*100)
    return percentile_

## Interest rates - NAICS

In [0]:
interest_df = lend_df[~lend_df["InitialInterestRate"].isnull()][['BorrName', 'InitialInterestRate', 'ApprovalDate', 'FirstDisbursementDate', 'code', 'desc']]

In [0]:
interest_group = interest_df[['desc', 'code', 'InitialInterestRate']].groupby(['desc','code'], as_index=False).agg(
                      {'InitialInterestRate':['mean','std', 'min', 'max', 'median', percentile(0.25), percentile(0.75),'count']})

In [0]:
interest_group[interest_group[('InitialInterestRate','count')] > 50].sort_values([('InitialInterestRate','mean')])

desc    code  \
                                                                              
800                                              Private Households  814110   
208                  Conveyor and Conveying Equipment Manufacturing  333922   
722                               Packaging Machinery Manufacturing  333993   
918   Special Die and Tool, Die Set, Jig, and Fixture Manufacturing  333514   
299                  Fabricated Pipe and Pipe Fitting Manufacturing  332996   
...                                                             ...     ...   
25                             All Other General Merchandise Stores  452319   
1026        Wireless Telecommunications Carriers (except Satellite)  517312   
60                  All Other Support Activities for Transportation  488999   
557                                      Musical Groups and Artists  711130   
282                       Electronic Shopping and Mail-Order Houses  454110   

     InitialInterestRate                                             \
                    mean    std    min     max median percentile_25   
800               5.5360 0.8624 2.1000  9.7500 5.5000        4.9500   
208               5.6085 1.3252 2.7500  9.7500 5.5000        4.7500   
722               5.6737 1.5011 2.7500  9.7500 5.5000        4.9425   
918               5.6933 1.2343 0.0000 11.1300 5.7300        5.0000   
299               5.6968 1.2940 3.5000 11.2500 5.5000        4.7500   
...                  ...    ...    ...     ...    ...           ...   
25                7.5855 1.6929 4.4000 12.0000 7.0000        6.5000   
1026              7.6020 1.4701 5.2500 11.5000 7.2500        6.7500   
60                7.6093 1.8680 2.7500 13.5000 7.6000        6.0000   
557               7.8787 1.6635 5.0000 12.0000 7.7500        6.4500   
282               8.7494 1.8594 1.9900 12.0000 8.7400        7.5000   

                          
     percentile_75 count  
800         6.0000   126  
208         6.5000    72  
722         6.2500    60  
918         6.1300   261  
299         6.1500    66  
...            ...   ...  
25          8.2500   131  
1026        8.2500    65  
60          9.0500   478  
557         9.1500   140  
282        10.3800  3375  

[651 rows x 10 columns]

In [0]:
# Save to CSV
interest_group[interest_group[('InitialInterestRate','count')] > 50].sort_values([('InitialInterestRate','mean')]).to_csv('interest_summ_by_group.csv')

## Interest rates - Lender

In [0]:
interest_df = lend_df[~lend_df["InitialInterestRate"].isnull()][['InitialInterestRate', 'BankName']]

In [0]:
interest_group = interest_df[['BankName', 'InitialInterestRate']].groupby(['BankName'], as_index=False).agg(
                      {'InitialInterestRate':['mean','std', 'min', 'max', 'median', percentile(0.25), percentile(0.75),'count']})

In [0]:
interest_group[interest_group[('InitialInterestRate','count')] > 50].sort_values([('InitialInterestRate','mean')])

BankName InitialInterestRate                \
                                                     mean    std    min   
1706                      Landmark CU              4.3279 0.8118 1.9900   
1491                    HawaiiUSA FCU              4.5571 1.0629 2.2500   
2554                  St. Mary's Bank              4.6226 1.2139 2.9900   
206          BIZCapital BIDCO II, LLC              4.7294 0.7197 3.2500   
1459  Grow America Fund, Incorporated              4.7336 1.5903 1.0000   
...                               ...                 ...    ...    ...   
692     Colorado Lending Source, Ltd.              9.0061 0.7597 4.7500   
817                            Craft3              9.2754 0.7365 6.0000   
421          Brooklyn Cooperative FCU              9.5588 1.3081 5.5000   
41                             Accion              9.6293 0.8524 5.5000   
216    BSD Capital, LLC dba Lendistry             10.1457 0.5419 8.2500   

                                                        
         max  median percentile_25 percentile_75 count  
1706  6.5000  4.2500        4.0000        4.5975    58  
1491  6.7500  5.2500        3.0000        5.2500   302  
2554  8.0000  4.2500        3.9900        5.2500   176  
206   6.2500  4.7600        4.1100        5.2450    51  
1459  9.0000  4.7500        3.7500        5.7600   305  
...      ...     ...           ...           ...   ...  
692  10.0000  9.0000        8.5000        9.6875    82  
817  11.0000  9.2500        9.2500        9.5000    59  
421  12.0000  9.7500        9.0000        9.7500   276  
41   11.5000  9.5000        9.2500       10.0000   116  
216  11.2500 10.0000        9.7500       10.5000   151  

[894 rows x 9 columns]

In [0]:
# Save to CSV
interest_group[interest_group[('InitialInterestRate','count')] > 50].sort_values([('InitialInterestRate','mean')]).to_csv('interest_summ_by_lender.csv')

## Interest rates - Term

In [0]:
interest_df = lend_df[~lend_df["InitialInterestRate"].isnull()][['InitialInterestRate', 'TermInMonths']]

In [0]:
interest_group = interest_df[['TermInMonths', 'InitialInterestRate']].groupby(['TermInMonths'], as_index=False).agg(
                      {'InitialInterestRate':['mean','std', 'min', 'max', 'median', percentile(0.25), percentile(0.75),'count']})

In [0]:
interest_group.sort_values([('InitialInterestRate','mean')])

TermInMonths InitialInterestRate                               \
                                mean    std    min     max median   
331          377              3.1800    nan 3.1800  3.1800 3.1800   
333          847              4.5000    nan 4.5000  4.5000 4.5000   
332          720              4.7500    nan 4.7500  4.7500 4.7500   
326          328              5.1250 1.2374 4.2500  6.0000 5.1250   
327          332              5.2500    nan 5.2500  5.2500 5.2500   
..           ...                 ...    ...    ...     ...    ...   
55            55              7.0225 1.7668 2.5000 11.7400 6.7500   
68            68              7.0456 1.7098 3.2500 11.5000 6.5000   
84            84              7.0551 1.7232 0.0000 12.0000 6.7500   
70            70              7.0811 1.7119 1.5000 12.0000 6.7500   
67            67              7.0899 1.7319 2.5000 11.5000 6.7500   

                                         
    percentile_25 percentile_75   count  
331        3.1800        3.1800       1  
333        4.5000        4.5000       1  
332        4.7500        4.7500       1  
326        4.6875        5.5625       2  
327        5.2500        5.2500       1  
..            ...           ...     ...  
55         5.7500        8.2500     376  
68         5.9000        8.2400     469  
84         5.7500        8.3500  161068  
70         6.0000        8.0000     465  
67         5.7500        8.4500     461  

[334 rows x 9 columns]

In [0]:
# Save to CSV
interest_group.sort_values([('InitialInterestRate','mean')]).sort_values([('InitialInterestRate','mean')]).to_csv('interest_summ_by_term.csv')

## Volume - NAICS

In [0]:
amount_df = lend_df[~lend_df["GrossApproval"].isnull()][['BorrName', 'GrossApproval', 'ApprovalDate', 'FirstDisbursementDate', 'code', 'desc']]

In [0]:
amount_group = amount_df[['desc', 'code', 'GrossApproval']].groupby(['desc','code'], as_index=False).agg(
                      {'GrossApproval':['mean','std', 'min', 'max', 'median', percentile(0.25), percentile(0.75),'count']})

In [0]:
amount_group[amount_group[('GrossApproval','count')] > 50].sort_values([('GrossApproval','mean')])

desc  \
                                                                          
117                                               Blood and Organ Banks   
662  Other Direct Insurance (except Life, Health, and Medical) Carriers   
348                                   Footwear and Leather Goods Repair   
82                                              Apprenticeship Training   
564                                          Musical Groups and Artists   
..                                                                  ...   
834                                Refrigerated Warehousing and Storage   
89                           Assisted Living Facilities for the Elderly   
387                           Grain and Field Bean Merchant Wholesalers   
480                    Lessors of Miniwarehouses and Self-Storage Units   
418                            Hotels (except Casino Hotels) and Motels   

       code  GrossApproval                                               \
                      mean            std   min      max         median   
117  621991    34,400.0000    93,216.4088  5000   519000     5,000.0000   
662  524128    55,988.0415   284,596.6007  5000  5000000    20,000.0000   
348  811430    63,354.2601   168,807.8140   500  2035000    26,000.0000   
82   611513    63,380.6707   187,124.8777  5000  1250000    10,000.0000   
564  711130    73,902.2857   190,123.3163  3000  1625400    25,000.0000   
..      ...            ...            ...   ...      ...            ...   
834  493120   846,945.3623 1,112,450.5131  1000  4918500   420,050.0000   
89   623312   856,064.4105 1,022,772.0658  5000  5000000   520,000.0000   
387  424510   922,709.8971 1,153,501.8282  4800  5000000   510,000.0000   
480  531130 1,258,816.7249 1,324,345.7407  5000  5000000   758,650.0000   
418  721110 1,434,118.8210 1,233,655.8382  2000  5000000 1,134,800.0000   

                                         
    percentile_25  percentile_75  count  
117    5,000.0000    10,000.0000     90  
662    5,000.0000    35,000.0000    337  
348   12,750.0000    60,050.0000    223  
82     5,000.0000    50,000.0000     82  
564   10,000.0000    50,000.0000    350  
..            ...            ...    ...  
834  100,000.0000 1,074,787.5000    138  
89   200,000.0000 1,027,500.0000   1732  
387  120,250.0000 1,500,000.0000    175  
480  316,425.0000 1,725,000.0000   1214  
418  500,000.0000 2,000,000.0000  15681  

[798 rows x 10 columns]

In [0]:
# Save to CSV
amount_group[amount_group[('GrossApproval','count')] > 50].sort_values([('GrossApproval','mean')]).to_csv('amount_summ_by_group.csv')

## Volume - Lender

In [0]:
amount_df = lend_df[~lend_df["GrossApproval"].isnull()][['GrossApproval', 'BankName']]

In [0]:
amount_group = amount_df[['BankName', 'GrossApproval']].groupby(['BankName'], as_index=False).agg(
                      {'GrossApproval':['mean','std', 'min', 'max', 'median', percentile(0.25), percentile(0.75),'count']})

In [0]:
amount_group[amount_group[('GrossApproval','count')] > 50].sort_values([('GrossApproval','mean')])

BankName  GrossApproval                         \
                                               mean            std     min   
177   American Express National Bank    19,810.2679     6,033.2362   10000   
4774                VelocitySBA, LLC    20,078.2332   135,938.9459    5000   
667         Brooklyn Cooperative FCU    26,302.7138    23,768.5794    1500   
2296                   HawaiiUSA FCU    32,231.4570    31,929.9343    4900   
4005                    Teachers FCU    33,269.0909    37,404.0841    5000   
...                              ...            ...            ...     ...   
4709             United Pacific Bank 1,598,706.5714 1,456,747.9634   50000   
3977    T Bank, National Association 1,674,884.9515 1,360,254.1242  150000   
3614                      Savoy Bank 1,774,671.5278 1,453,659.7959   50000   
429                   Bank of George 2,028,105.0459 1,203,697.7753   75000   
621          BizCapital BIDCO I, LLC 2,228,110.6900 1,250,950.0183  200000   

                                                                   
          max         median  percentile_25  percentile_75  count  
177     50000    20,000.0000    15,000.0000    25,000.0000    224  
4774  4119500    10,000.0000     5,000.0000    12,500.0000  10833  
667    200000    15,677.0000    10,000.0000    38,125.0000    276  
2296   200000    21,000.0000    12,000.0000    42,075.0000    302  
4005   250000    25,000.0000    15,000.0000    40,000.0000     55  
...       ...            ...            ...            ...    ...  
4709  5000000 1,000,000.0000   471,250.0000 2,270,000.0000     84  
3977  5000000 1,090,000.0000   562,700.0000 2,519,500.0000    206  
3614  5000000 1,307,500.0000   548,750.0000 2,877,750.0000    144  
429   5000000 1,852,500.0000 1,043,750.0000 2,898,000.0000    218  
621   5000000 2,000,000.0000 1,400,000.0000 2,884,250.0000    100  

[1286 rows x 9 columns]

In [0]:
# Save to CSV
amount_group[amount_group[('GrossApproval','count')] > 50].sort_values([('GrossApproval','mean')]).to_csv('amount_summ_by_lender.csv')

## Volume - Term

In [0]:
amount_df = lend_df[~lend_df["GrossApproval"].isnull()][['GrossApproval', 'TermInMonths']]

In [0]:
amount_group = amount_df[['TermInMonths', 'GrossApproval']].groupby(['TermInMonths'], as_index=False).agg(
                      {'GrossApproval':['mean','std', 'min', 'max', 'median', percentile(0.25), percentile(0.75),'count']})

In [0]:
amount_group.sort_values([('GrossApproval','mean')])

TermInMonths  GrossApproval                                   \
                           mean            std      min      max   
341          346    20,000.0000            nan    20000    20000   
352          364    35,000.0000            nan    35000    35000   
344          349    35,000.0000            nan    35000    35000   
383          847    46,000.0000            nan    46000    46000   
373          430    46,000.0000            nan    46000    46000   
..           ...            ...            ...      ...      ...   
321          321 2,259,157.8947 1,625,432.2971   100000  4800000   
315          315 2,430,665.5833 1,385,330.7147    35000  5000000   
318          318 2,588,893.4453 1,483,984.3302   136000  5000000   
324          324 2,613,396.9811 1,733,922.6989    94000  5000000   
347          352 3,375,000.0000            nan  3375000  3375000   

                                                        
            median  percentile_25  percentile_75 count  
341    20,000.0000    20,000.0000    20,000.0000     1  
352    35,000.0000    35,000.0000    35,000.0000     1  
344    35,000.0000    35,000.0000    35,000.0000     1  
383    46,000.0000    46,000.0000    46,000.0000     1  
373    46,000.0000    46,000.0000    46,000.0000     1  
..             ...            ...            ...   ...  
321 2,000,000.0000   692,500.0000 3,738,500.0000    19  
315 2,227,500.0000 1,227,750.0000 3,314,400.0000    84  
318 2,367,500.0000 1,377,475.0000 3,820,500.0000   128  
324 2,444,000.0000 1,344,400.0000 4,250,000.0000    53  
347 3,375,000.0000 3,375,000.0000 3,375,000.0000     1  

[384 rows x 9 columns]

In [0]:
# Save to CSV
amount_group.sort_values([('GrossApproval','mean')]).to_csv('amount_summ_by_term.csv')

In [0]:
# Correlation amount with length of loan
lend_df['GrossApproval'].corr(lend_df['TermInMonths'])

0.5063744579165977

#### Volume - ANOVA Testing

In [0]:
from statsmodels.formula.api import ols

In [0]:
results = ols('GrossApproval ~ C(desc)', data=amount_df).fit()

In [0]:
results.summary()
# TODO

# 3. How to determine if company is still around

In [0]:
find_active = lend_df[['BorrName', 'BorrStreet', 'BorrCity', 'BorrState', 'BorrZip', 'ApprovalFiscalYear', 'LoanStatus', 'GrossChargeOffAmount', 'JobsSupported']].sort_values('ApprovalFiscalYear')

In [0]:
find_active.to_csv("business_identities_by_year.csv")

In [0]:
# TODO